<a href="https://colab.research.google.com/github/kimtoto23/Network_Simulation_CMCP/blob/main/network_simulation_gurobipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connected Maximum Coverage Problem using GA Algorithm

##### google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Define the sets and data
I = range(12)  # Facility candidates set
J = range(12)  # Facility demand set
cost = [
    [0, 15, 37, 55, 24, 60, 18, 33, 48, 40, 58, 67],
    [15, 0, 22, 40, 38, 52, 33, 48, 42, 55, 61, 61],
    [37, 22, 0, 18, 16, 30, 41, 28, 20, 58, 39, 39],
    [55, 40, 18, 0, 34, 12, 59, 46, 24, 62, 43, 34],
    [24, 38, 16, 34, 0, 36, 25, 12, 24, 47, 37, 43],
    [60, 52, 30, 12, 36, 0, 57, 42, 12, 50, 31, 22],
    [18, 33, 41, 59, 25, 57, 0, 15, 45, 22, 40, 61],
    [33, 48, 28, 46, 12, 42, 15, 0, 30, 37, 25, 46],
    [48, 42, 20, 24, 24, 12, 45, 30, 0, 38, 19, 19],
    [40, 55, 58, 62, 47, 50, 22, 37, 38, 0, 19, 40],
    [58, 61, 39, 43, 37, 31, 40, 25, 19, 19, 0, 21],
    [67, 61, 39, 34, 43, 22, 61, 46, 19, 40, 21, 0]
]

# Node demands
node_edge = pd.DataFrame({"node": [0,1,2,3,4,5,6,7,8,9,10,11], "demand":[15,10,12,18,5,24,11,16,13,22,19,20]})
demand = node_edge['demand']

# Create a Gurobi model
model = gp.Model("Maximum_Coverage_Facility_Location")

# Decision variables
x = {}  # Facility selection variables
y = {}  # Coverage variables
a = {}  # Binary decision variable for covering
for i in I:
    x[i] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}")
    for j in J:
        y[i, j] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}_{j}")
        a[i, j] = model.addVar(vtype=GRB.BINARY, name=f"a_{i}_{j}")

# Objective function
model.setObjective(gp.quicksum(demand[j] * y[i, j] for i in I for j in J), sense=GRB.MAXIMIZE)

# Constraints
for i in I:
    model.addConstr(gp.quicksum(cost[j][i] * y[i, j] for j in J) <= 30 * x[i], name=f"Distance_Constraint_{i}")

for i in I:
    for j in J:
        model.addConstr(y[i, j] <= a[i, j] * x[j], name=f"Coverage_Constraint_{i}_{j}")

model.addConstr(gp.quicksum(x[i] for i in I) == 3, name="Facility_Count_Constraint")

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found!")
    for i in I:
        if x[i].x > 0.5:
            print(f"Facility {i} is selected.")
            covered_nodes = [j for j in J if y[i, j].x > 0.5]
            print(f"Nodes covered by Facility {i}: {covered_nodes}")
else:
    print("No optimal solution found.")
